In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [2]:
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#config = tf.compat.v1.ConfigProto(device_count = {'GPU': 1})

# To Do

## Preliminary work on the dataset

In [3]:
yahoo_df = pd.read_csv("../data/NSQ.csv")
yahoo_df = yahoo_df.set_index("Date")
yahoo_df = yahoo_df.astype(float,errors="raise")
yahoo_df

Open     Close          vwap          SO     ROC_1     ROC_2  \
Date                                                                           
2001-04-24   1822.00   1791.00   1670.174447   71.689895 -1.539307 -8.318403   
2001-04-25   1785.00   1824.00   1695.983094   77.439024  1.842546  0.274876   
2001-04-26   1816.50   1774.00   1720.324225   63.103803 -2.741228 -0.949190   
2001-04-27   1776.00   1823.00   1737.458814   73.175745  2.762120 -0.054825   
2001-04-30   1825.50   1866.50   1762.278372   80.837004  2.386177  5.214205   
...              ...       ...           ...         ...       ...       ...   
2022-03-22  14390.00  14654.00  13783.466184  100.000000  1.972791  3.963662   
2022-03-23  14654.00  14447.00  13793.821985   87.130867 -1.412584  0.532341   
2022-03-24  14462.75  14763.75  13839.410958  100.000000  2.192497  0.748942   
2022-03-25  14765.00  14755.75  13905.922231   99.534410 -0.054187  2.137122   
2022-03-28  14756.00  14985.25  14039.538800  100.000000  1.555326  1.500296   

               ROC_3     ROC_4     ROC_5      ROC_6  ...   SRSI_10   SRSI_14  \
Date                                                 ...                       
2001-04-24 -8.318403 -2.636586  7.085202  10.012285  ...  0.703108  0.856380   
2001-04-25 -6.629127 -6.629127 -0.842620   9.058296  ...  0.662475  0.901004   
2001-04-26 -2.473887 -9.188636 -9.188636  -3.560750  ...  0.584225  0.770925   
2001-04-27  1.786711  0.219901 -6.680317  -6.680317  ...  0.568203  0.854340   
2001-04-30  2.330044  4.215522  2.611325  -4.453545  ...  0.705601  0.860369   
...              ...       ...       ...        ...  ...       ...       ...   
2022-03-22  3.842543  5.024009  8.937499  12.329922  ...  0.747822  0.523177   
2022-03-23  2.495088  2.375680  3.540457   7.398666  ...  0.640338  0.523453   
2022-03-24  2.736509  4.742290  4.620263   5.810578  ...  0.717648  0.591346   
2022-03-25  0.694350  2.680839  4.685533   4.563573  ...  0.802474  0.691443   
2022-03-28  3.725687  2.260475  4.277861   6.313735  ...  0.903672  0.722857   

             SRSI_30  Williams_14      ATR_14         CCI         APO  \
Date                                                                    
2001-04-24  0.921375   -32.389937  129.530633   67.404009   58.849359   
2001-04-25  0.875157   -28.130081  123.659882   62.777426   86.003205   
2001-04-26  0.858921   -39.821429  119.305231   57.000693  103.608974   
2001-04-27  0.951489   -31.182796  111.664534   55.239375  114.083333   
2001-04-30  1.000000   -26.100000  109.509262   71.817882  125.823718   
...              ...          ...         ...         ...         ...   
2022-03-22  0.414869    -2.103907  430.511541  134.641163 -196.954058   
2022-03-23  0.379605   -13.954487  405.276669  113.361254 -110.561430   
2022-03-24  0.380847    -0.328632  396.073113  117.456985    8.235045   
2022-03-25  0.411403    -4.468758  378.996698  113.314284   87.327994   
2022-03-28  0.480099    -0.572333  373.563805  119.113994  172.374468   

            Differenced  Previous_differenced  principalDf  
Date                                                        
2001-04-24    -0.015393             -0.068851    -0.045527  
2001-04-25     0.018425             -0.015393    -1.632587  
2001-04-26    -0.027412              0.018425     2.965141  
2001-04-27     0.027621             -0.027412    -0.184117  
2001-04-30     0.023862              0.027621    -3.923256  
...                 ...                   ...          ...  
2022-03-22     0.019728              0.019524    -9.177853  
2022-03-23    -0.014126              0.019728    -4.593364  
2022-03-24     0.021925             -0.014126    -6.861921  
2022-03-25    -0.000542              0.021925    -6.255217  
2022-03-28     0.015553             -0.000542    -8.056809  

[5290 rows x 51 columns]

# Train, test, val

In [4]:
X = yahoo_df[['ROC_30', '4 Day ROI', 'EMA_12', 'MACD_12_26_9', 'SRSI_30',
       'Williams_14', 'ATR_14', 'Previous_differenced']]
y = yahoo_df["Up down"]

In [5]:
# Split in 80/20 the dataframeX
X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, test_size = 0.20,
                                                          shuffle=False)

# Split in 75/25 the remaining 80 %
X_train, X_valid, y_train, y_valid = train_test_split(X_train_80, y_train_80,
                                                      test_size = 0.25, shuffle=False)

# Normalize

In [6]:
def normalize(df):
    #df = data
    for column in df:
        df[column]=((df[column]-df[column].mean())/df[column].std())
    return df

In [7]:
X_train_80 = normalize(X_train_80)
X_test = normalize(X_test)


In [8]:
X_train = normalize(X_train)
X_valid = normalize(X_valid)

# Light Gradient Boosting Machine


In [9]:
# Build LightGBM Model
valid_data = lgb.Dataset(X_valid, label = y_valid)
train_data = lgb.Dataset(X_train, label = y_train)
test_data = lgb.Dataset(X_train_80, label = y_train_80)

In [10]:
# Select Hyper-Parameters
params = {'boosting_type': 'dart',
          "num_leaves":[2],
          'max_depth' : [10],
          'objective': 'multiclass',
          'nthread': 4,
          'learning_rate': 0.0125,
          'max_bin': 100,
          'reg_lambda': 1.4,
          'num_class' : 2,  
          'metric' : 'multi_logloss'}

In [11]:
# Train the model
lgbm = lgb.train(params, train_data)

# Predict
pred_train = lgbm.predict(X_train)
pred_valid = lgbm.predict(X_valid)

y_pred_train = np.argmax(pred_train, axis = 1)
y_pred_valid = np.argmax(pred_valid, axis = 1)

# Compute accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000198 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 800
[LightGBM] [Info] Number of data points in the train set: 3174, number of used features: 8
[LightGBM] [Info] Start training from score -0.766335
[LightGBM] [Info] Start training from score -0.624953
Train Accuracy: 0.53529 - Validation Accuracy: 0.55860


# Tuning very slow

In [12]:

param_test ={
            'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 100

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

gs.fit(X_train, y_train)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

In [13]:
params = {'boosting_type': 'dart',
          "learning_rate":0.001,
          "num_iterations":1000,
        'colsample_bytree': 0.9242593373375023,
          'min_child_samples': 282,
          'min_child_weight': 10.0, 
          'num_leaves': 36, 
          'reg_alpha': 0, 
          'reg_lambda': 0.1, 
          'subsample': 0.4955174534902478,
          'objective': 'multiclass',
          'metric' : 'multi_logloss',
          'num_class' : 2, #Away, Draw and Home,
         }

In [14]:
len(pred_train)

3174

In [15]:
# Train the model
lgbm = lgb.train(params, train_data)

# Predict
pred_train = lgbm.predict(X_train)
pred_valid = lgbm.predict(X_valid)

y_pred_train = np.argmax(pred_train, axis = 1)
y_pred_valid = np.argmax(pred_valid, axis = 1)

# Compute accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000096 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 800
[LightGBM] [Info] Number of data points in the train set: 3174, number of used features: 8
[LightGBM] [Info] Start training from score -0.766335
[LightGBM] [Info] Start training from score -0.624953
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

C:\Users\enric\anaconda3\envs\data_science\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [16]:
# Predict
pred_test = lgbm.predict(X_test)

y_pred_test = np.argmax(pred_test, axis = 1)

# Compute accuracy
test_acc = accuracy_score(y_test, y_pred_test)
print("Test Accuracy: {:.5f}".format(test_acc))

Test Accuracy: 0.52930


# Save accuracies

In [17]:
# Train the test set

# Predict
pred = lgbm.predict(X_test)
y_pred = np.argmax(pred, axis = 1)

# Compute accuracy
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.5f}".format(acc))

# Store the test and the validation accuracy
LGBM = acc

Test Accuracy: 0.52930


In [18]:
df_prediction = pd.read_csv("../data/model_accuracy_nsq.csv", index_col = False)
df_prediction =  df_prediction.drop(columns=["Unnamed: 0"])

In [19]:
#df_prediction = pd.read_csv("../data/model_accuracy.csv", index_col = False)
df_prediction["LGBM_pred"] = y_pred
df_prediction["LGBM_accuracy"] = [LGBM for x in range(len(y_pred))]
df_prediction["LGBM_prob"] = pred[:,1]
df_prediction

RF_pred  RF_accuracy   RF_prob  SVC_pred  SVC_accuracy  SVC_prob  \
0           1     0.523629  0.657543         1      0.558601  0.544082   
1           0     0.523629  0.324182         1      0.558601  0.544076   
2           0     0.523629  0.496864         1      0.558601  0.545460   
3           1     0.523629  0.574333         1      0.558601  0.545009   
4           0     0.523629  0.421569         1      0.558601  0.543872   
...       ...          ...       ...       ...           ...       ...   
1053        1     0.523629  0.679418         1      0.558601  0.539406   
1054        1     0.523629  0.555801         1      0.558601  0.537904   
1055        1     0.523629  0.573323         1      0.558601  0.540966   
1056        1     0.523629  0.719503         1      0.558601  0.536726   
1057        1     0.523629  0.531916         1      0.558601  0.541307   

      XGB_pred  XGB_accuracy  XGB_prob  ANN_pred  ANN_accuracy  ANN_prob  \
0            1      0.533081  0.536098         0      0.441399  0.464561   
1            0      0.533081  0.390463         0      0.441399  0.464505   
2            1      0.533081  0.552227         0      0.441399  0.464505   
3            1      0.533081  0.596263         0      0.441399  0.464505   
4            0      0.533081  0.483318         0      0.441399  0.464505   
...        ...           ...       ...       ...           ...       ...   
1053         1      0.533081  0.598255         0      0.441399  0.464011   
1054         1      0.533081  0.567730         0      0.441399  0.464421   
1055         1      0.533081  0.561254         0      0.441399  0.463568   
1056         1      0.533081  0.590790         0      0.441399  0.464013   
1057         1      0.533081  0.569102         0      0.441399  0.462834   

      LGBM_pred  LGBM_accuracy  LGBM_prob  
0             1       0.529301   0.532322  
1             1       0.529301   0.518503  
2             1       0.529301   0.515129  
3             1       0.529301   0.531008  
4             1       0.529301   0.529314  
...         ...            ...        ...  
1053          1       0.529301   0.510938  
1054          1       0.529301   0.511045  
1055          1       0.529301   0.509250  
1056          1       0.529301   0.510938  
1057          1       0.529301   0.511825  

[1058 rows x 15 columns]

In [20]:
df_prediction.to_csv("../data/model_accuracy_nsq.csv")
